## Working with Tilecode in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/15_tilecode.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/15_tilecode.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/15_tilecode.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/15_tilecode.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/15_tilecode.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2tilecode

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2tilecode

lat = 10.775276
lon = 106.706797
res = 14
tilecode_id = latlon2tilecode(lat, lon, res)
tilecode_id

### Tilecode to Polygon

In [ ]:
from vgrid.conversion.dggs2geo.tilecode2geo import tilecode2geo

tilecode_geo = tilecode2geo(tilecode_id)
tilecode_geo

### Tilecode to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.tilecode2geo import tilecode2geojson

tilecode_geojson = tilecode2geojson(tilecode_id)
tilecode_geojson

### Vector to Tilecode

In [ ]:
from vgrid.conversion.vector2dggs.vector2tilecode import vector2tilecode

file_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
vector_to_tilecode = vector2tilecode(
    file_path,
    compact=False,
    resolution=18,
    topology=True,
    predicate="intersects",
    output_format="gpd",
)
# Visualize the output

vector_to_tilecode.plot(edgecolor="white")

### Tilecode Compact

In [ ]:
from vgrid.conversion.dggscompact.tilecodecompact import tilecodecompact

tilecode_compacted = tilecodecompact(
    vector_to_tilecode, tilecode_id="tilecode", output_format="gpd"
)
tilecode_compacted.plot(edgecolor="white")

### Tilecode Expand

In [ ]:
from vgrid.conversion.dggscompact.tilecodecompact import tilecodeexpand

tilecode_expanded = tilecodeexpand(
    vector_to_tilecode, resolution=19, output_format="gpd"
)
tilecode_expanded.plot(edgecolor="white")

### Tilecode Binning

In [ ]:
from vgrid.binning.tilecodebin import tilecodebin

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
tilecode_bin = tilecodebin(
    file_path,
    resolution=18,
    stats=stats,
    # numeric_field="confidence",
    # category="category",
    output_format="gpd",
)
tilecode_bin.plot(
    column=stats,  # numeric column to base the colors on
    cmap="Spectral_r",  # color scheme (matplotlib colormap)
    legend=True,
    linewidth=0.2,  # boundary width (optional)
)

### Raster to Tilecode

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, "r")
print(src.meta)
show(src)

#### Convert raster to Tilecode

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2tilecode import raster2tilecode

raster_to_tilecode = raster2tilecode(raster_file, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

tilecode_layer = folium.GeoJson(
    raster_to_tilecode,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["tilecode", "band_1", "band_2", "band_3"],
        aliases=["Tilecode ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(tilecode_layer.get_bounds())

# Display the map
m

### Tilecode Generator

In [ ]:
from vgrid.generator.tilecodegrid import tilecodegrid

# tilecode_grid = tilecodegrid(resolution=2,output_format="gpd")
tilecode_grid = tilecodegrid(
    resolution=18,
    bbox=[106.699007, 10.762811, 106.717674, 10.778649],
    output_format="gpd",
)
tilecode_grid.plot(edgecolor="white")

### Tilecode Inspect

In [ ]:
from vgrid.stats.tilecodestats import tilecodeinspect

resolution = 7
tilecode_inspect = tilecodeinspect(resolution)
tilecode_inspect.head()

### Tilecode Normalized Area Histogram

In [ ]:
from vgrid.stats.tilecodestats import tilecode_norm_area_hist

tilecode_norm_area_hist(tilecode_inspect)

### Distribution of Tilecode Area Distortions

In [ ]:
from vgrid.stats.tilecodestats import tilecode_norm_area

tilecode_norm_area(tilecode_inspect)

### Tilecode IPQ Compactness Histogram

Isoperimetric Inequality (IPQ) Compactness (suggested by [Osserman, 1978](https://sites.math.washington.edu/~toro/Courses/20-21/MSF/osserman.pdf)):

$$C_{IPQ} = \frac{4 \pi A}{p^2}$$
The range of the IPQ compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.tilecodestats import tilecode_compactness_ipq_hist

tilecode_compactness_ipq_hist(tilecode_inspect)

### Distribution of Tilecode IPQ Compactness

In [ ]:
from vgrid.stats.tilecodestats import tilecode_compactness_ipq

tilecode_compactness_ipq(tilecode_inspect)

### Tilecode Convex hull Compactness Histogram:

$$C_{CVH} = \frac{A}{A_{CVH}}$$


The range of the convex hull compactness metric is [0,1]. 

As shapes become more concave, their convex hull compactness decreases toward 0.

In [ ]:
from vgrid.stats.tilecodestats import tilecode_compactness_cvh_hist
tilecode_compactness_cvh_hist(tilecode_inspect)

### Distribution of Tilecode Convex hull Compactness

In [ ]:
from vgrid.stats.tilecodestats import tilecode_compactness_cvh
tilecode_compactness_cvh(tilecode_inspect)

### Tilecode Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats import tilecodestats

tilecodestats("km")